In [1]:
import sys
import torch
import tqdm
import numpy as np
import random
import os
import json
sys.path.append('../../')

In [2]:
from models.cnn.search_cnn import  SearchCNN, SearchCNNController
from configobj import ConfigObj

In [3]:
import utils
# get data with meta info
input_size, input_channels, n_classes, train_data, valid_data = utils.get_data(
    'mnist', '../../data/', cutout_length=0, validation=True)

train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=64,
                                               shuffle=True,
                                               num_workers=32,
                                               pin_memory=True)

valid_loader = torch.utils.data.DataLoader(valid_data,
                                           batch_size=64,
                                           shuffle=False,
                                           num_workers=1,
                                           pin_memory=True)

/home/local/AP-TEAM/bakhteev/oleg_torch/lib/python3.8/site-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 24, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [16]:
ckp_path = '../../searchs/mnist_darts/best_{}.pth.tar' # это шаблон названия сохраненных моделей
basecfg_path = '../../configs/mnist/darts.cfg'  #конфиг, на который мы ориентируемся при загрузки модели
seeds = [0, 13, 21, 42, 99]  # сиды. по идее, можно брать из конфига
cfg = ConfigObj(basecfg_path)
name = cfg['name'] # имя для сохранения результатов
fine_tune = True # надо ли дообучать модели
if fine_tune:
    name +='_fine'

In [6]:
def fix_structure(sc): # во время тестов нашей модели нужно перейти от непрерывной структуры к one-hot
    for alpha in sc.alpha_reduce:
        alpha.requires_grad = False
        for subalpha in alpha:
            argm = torch.argmax(subalpha)
            subalpha.data*=0
            subalpha.data[argm] += 1
            
            
            

In [18]:
if fine_tune:    
    # создаем модель с обязательным указанием, что к структуре не применяется softmax
    cfg = ConfigObj(basecfg_path)
    cfg['darts']['sampling_mode'] = 'naive' 
    cfg['device'] = 'cuda'
    for s in seeds:
        print (s)
        sc = SearchCNNController(**cfg)
        sc.load_state_dict(torch.load(ckp_path.format(s)))
        fix_structure(sc)
        sc = sc.to('cuda')
        optim = torch.optim.Adam(sc.weights())
        correct = 0
        total = 0
        # дообучаем одну эпоху
        for x,y in tqdm.tqdm(train_loader):
            x = x.cuda()
            y = y.cuda()            
            optim.zero_grad()
            loss = sc.loss(x,y)
            loss.backward()
            optim.step()                              
        torch.save(sc.state_dict(), ckp_path.format(s)+'.fine')

0


  0%|          | 0/938 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 24, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 938/938 [00:49<00:00, 19.12it/s]


13


100%|██████████| 938/938 [00:49<00:00, 18.99it/s]


21


100%|██████████| 938/938 [00:49<00:00, 18.91it/s]


42


100%|██████████| 938/938 [00:48<00:00, 19.23it/s]


99


100%|██████████| 938/938 [00:49<00:00, 18.79it/s]


In [19]:
# смотрим качество модели, которое мы получили на обучении
cfg = ConfigObj(basecfg_path)
cfg['device'] = 'cuda'
for s in seeds:
    print (s)
    sc = SearchCNNController(**cfg)
    if fine_tune:
        sc.load_state_dict(torch.load(ckp_path.format(s)+'.fine'))
    else:
        sc.load_state_dict(torch.load(ckp_path.format(s)))
    sc = sc.to('cuda')
    sc.eval()
    correct = 0
    total = 0
    for x,y in tqdm.tqdm(valid_loader):
        x = x.cuda()
        y = y.cuda()
        out = sc(x)
        correct += torch.eq(torch.argmax(out, 1), y).sum()
        total += len(x)
    print (correct*1.0/total*1.0)

0


100%|██████████| 157/157 [00:02<00:00, 59.19it/s]


tensor(0.1139, device='cuda:0')
13


100%|██████████| 157/157 [00:02<00:00, 60.64it/s]


tensor(0.1371, device='cuda:0')
21


...

tensor(0.1178, device='cuda:0')
42


100%|██████████| 157/157 [00:02<00:00, 60.31it/s]


tensor(0.1137, device='cuda:0')
99


100%|██████████| 157/157 [00:02<00:00, 60.03it/s]

tensor(0.2067, device='cuda:0')


In [20]:
# смотрим качество модели при условии, что мы делаем структуру дискретной
# в DARTS качество падает
cfg = ConfigObj(basecfg_path)
cfg['darts']['sampling_mode'] = 'naive'
cfg['device'] = 'cuda'
for s in seeds:
    print (s)
    sc = SearchCNNController(**cfg)
    if fine_tune:
        sc.load_state_dict(torch.load(ckp_path.format(s)+'.fine'))
    else:
        sc.load_state_dict(torch.load(ckp_path.format(s)))
    fix_structure(sc)    
    sc = sc.to('cuda')
    sc.eval()
    correct = 0
    total = 0
    for x,y in tqdm.tqdm(valid_loader):
        x = x.cuda()
        y = y.cuda()
        out = sc(x)
        correct += torch.eq(torch.argmax(out, 1), y).sum()
        total += len(x)
    print (correct*1.0/total*1.0)

0


100%|██████████| 157/157 [00:02<00:00, 59.68it/s]


tensor(0.9879, device='cuda:0')
13


100%|██████████| 157/157 [00:02<00:00, 56.76it/s]


tensor(0.9895, device='cuda:0')
21


...

tensor(0.9874, device='cuda:0')
42


100%|██████████| 157/157 [00:02<00:00, 57.76it/s]


tensor(0.9869, device='cuda:0')
99


100%|██████████| 157/157 [00:02<00:00, 57.29it/s]

tensor(0.9883, device='cuda:0')


In [21]:
# FGM-атака
# сохраняем результаты в json-файл 
results = []
cfg = ConfigObj(basecfg_path)
cfg['darts']['sampling_mode'] = 'naive'
cfg['device'] = 'cuda'
for s in seeds:
    results.append([])
    print (s)
    sc = SearchCNNController(**cfg)
    if fine_tune:
        sc.load_state_dict(torch.load(ckp_path.format(s)+'.fine'))
    else:
        sc.load_state_dict(torch.load(ckp_path.format(s)))
        
    fix_structure(sc)

    sc = sc.to('cuda')
    sc.eval()
    for eps in np.linspace(0.0, 1.0, 11):
        
        correct = 0
        total = 0

        for x,y in tqdm.tqdm(valid_loader):
            x = x.cuda()
            x.requires_grad = True                                                 
            y = y.cuda()
            out = sc(x)
            loss = sc.criterion(out, y)
            sc.zero_grad()
            loss.backward()
            data_grad = x.grad.data
            sign_data_grad = data_grad.sign()    
            perturbed_image = x + eps*sign_data_grad                    
            out = sc(perturbed_image)
            correct += torch.eq(torch.argmax(out, 1), y).sum().cpu().detach().numpy()
            total += len(x)
        print (eps, correct*1.0/total*1.0)
        results[-1].append((eps, float(correct*1.0/total*1.0)))
with open(name+'_fgm.json', 'w') as out:
    out.write(json.dumps(results))

0


  0%|          | 0/157 [00:00<?, ?it/s]

0.0 0.9879


  0%|          | 0/157 [00:00<?, ?it/s]

0.1 0.6777


...

0.8 0.0073


  0%|          | 0/157 [00:00<?, ?it/s]

0.9 0.0062


100%|██████████| 157/157 [00:08<00:00, 18.02it/s]

1.0 0.0067


In [22]:
# structure
cfg = ConfigObj(basecfg_path)
cfg['darts']['sampling_mode'] = 'naive'
cfg['device'] = 'cuda'
results = []
for s in seeds:
    print (s)
    sc = SearchCNNController(**cfg)
    if fine_tune:
        sc.load_state_dict(torch.load(ckp_path.format(s)+'.fine'))
    else:
        sc.load_state_dict(torch.load(ckp_path.format(s)))
    results.append([])
    fix_structure(sc)
    ones = []
    for i, alpha in enumerate(sc.alpha_reduce):
        for j, subalpha in enumerate(alpha):
            ones.append((i,j, torch.argmax(subalpha)))

    sc = sc.to('cuda')
    sc.eval()
    for eps in np.linspace(0.0, 1.0, 11):
        correct = 0
        total = 0

        for x,y in tqdm.tqdm(valid_loader):
            to_prune = random.sample(ones,  int(len(ones)*eps))
            for i, j, argm in to_prune:
                sc.alpha_reduce[i][j].data *= 0

            x = x.cuda()                                                          
            y = y.cuda()
            out = sc(x)            
            correct += torch.eq(torch.argmax(out, 1), y).sum().cpu().detach().numpy()
            total += len(x)
            for i, j, argm in to_prune:
                sc.alpha_reduce[i][j][argm].data += 1
        print (eps, correct*1.0/total*1.0)
        results.append( (eps, float(correct*1.0/total*1.0)))
with open(name+'_struct.json', 'w') as out:
    out.write(json.dumps(results))

0


  0%|          | 0/157 [00:00<?, ?it/s]

0.0 0.9879


  0%|          | 0/157 [00:00<?, ?it/s]

0.1 0.8342


...

0.8 0.1319


  0%|          | 0/157 [00:00<?, ?it/s]

0.9 0.1189


100%|██████████| 157/157 [00:02<00:00, 56.19it/s]

1.0 0.1032
